# Computational Programming with Python
### Lecture 12: Testing

### Center for Mathematical Sciences, Lund University
Lecturer: Claus Führer, Malin Christersson, Robert Klöfkorn


## Why tests?

- Because you do them anyway.
- Because it will keep your code alive.

## Automated tests

- Ensure a constant (high) quality standard of your code.
- Serve as a documentation of the use of your code.
- Document the test cases &rarr; test protocol

# In general

\begin{align}
\text{Unit } & \text{Testing}\\
\downarrow \\
\text{Integration} & \text{ Testing} \\
\downarrow \\
\text{System} & \text{ Testing} \\
\downarrow \\
\text{User Acceptance } & \text{Testing (UAT)}\\
\end{align}

## Example

Let's assume we want to test an implementation of the bisection algorithm:

In [1]:
from numpy import *

def bisect(f, a, b, tol=1.e-8):
    if f(a) * f(b) >0:
        raise ValueError("Incorrect initial [a, b]")   
    for i in range(100):
        c = (a + b) / 2.
        if f(a) * f(c) <= 0:
            b = c
        else:
            a = c
        if abs(a - b) < tol:
            return (a + b) / 2
    raise Exception('No root found')   

## Example (cont)

Let's assume we want to test an implementation of the bisection algorithm:

In [2]:
def test_identity():
    result = bisect(lambda x: x, -1., 1.) 
    expected = 0.
    assert allclose(result, expected),'expected zero not found'

test_identity()

## Example (cont)

Does the code handle wrong input correctly?

In [3]:
def test_badinput():
    try:
        bisect(lambda x: x, 0.5, 1)
    except ValueError:
        pass
    else:
        raise AssertionError()

test_badinput()

# Unittest Test Cases

We can use the testing framework `unittest` to bundle several test cases and automatize the testing.

### Simple example using Spyder
```python
import unittest

def my_function():
    return 0

class TestMyFunction(unittest.TestCase):
    def test_equal(self):
        self.assertEqual(my_function(), 1, "should be 0")
    def test_true(self):
        self.assertTrue(my_function() == 0, "should be 0")
        
if __name__ == '__main__':
    unittest.main()
```

### Simple example using Jupyter Notebook

In [4]:
import unittest

def my_function():
    return 0

class TestMyFunction(unittest.TestCase):
    def test_equal(self):
        self.assertEqual(my_function(), 0, "should be 0")
    def test_true(self):
        self.assertTrue(my_function() == 0, "should be 0")
        
unittest.main(argv=[''], verbosity=2, exit=False)  # we must write like this instead of if-statement

test_equal (__main__.TestMyFunction) ... ok
test_true (__main__.TestMyFunction) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


### Example testing bisection

```python
from bisection import bisect
import unittest

class TestIdentity(unittest.TestCase):
    def test(self):
        result = bisect(lambda x: x, -1.2, 1.)
        expected = 0.
        self.assertAlmostEqual(result, expected)
        
if __name__ == '__main__':
    unittest.main()
```

We recommend to group tests together.

```python
class TestIdentity(unittest.TestCase):
    def identity_fcn(self,x):
        return x
    def test_functionality(self):
        result = bisect(self.identity_fcn, -1.2, 1.)
        expected = 0.
        self.assertAlmostEqual(result, expected)
    def test_reverse_boundaries(self):
        result = bisect(self.identity_fcn, 1., -1.)
        expected = 0.
        self.assertAlmostEqual(result, expected)
    def test_exceeded_tolerance(self):
        tol=1.e-80
        self.assertRaises(Exception, bisect, self.identity_fcn, 
                          -1.2, 1.,tol)        
if __name__=='__main__':
       unittest.main()

```

### unittest.TestCase.assertRaises

Note the method `assertRaises`:

- `Exception`: the expected exception type
- `bisect`: the function to be called
- `self.identity_fcn, -1.2, 1., tol`: the parameters of this function

## Unittest: Preparing tests

Sometimes the execution of tests need som preparation. The special method `setUp` is executed first. The special method `tearDown` is executed after all tests.

```python
class TestFindInFile(unittest.TestCase):
    def setUp(self):
        file = open('test_file.txt', 'w')
        file.write('aha')
        file.close()
        self.file = open('test_file.txt', 'r')
    def tearDown(self):
        os.remove(self.file.name)
    def test_exists(self):
        line_no=find_string(self.file, 'aha')
        self.assertEqual(line_no, 0)
    def test_not_exists(self):
        self.assertRaises(NotFoundError, 
                          find_string, self.file, 'bha')

if __name__=='__main__':
                unittest.main() 

```

## Unittest: Preparing tests (cont.)

... and here the functions we want to test.

```python
class NotFoundError(Exception):
    pass

def find_string(file, string):
    for i,lines in enumerate(file.readlines()):
        if string in lines:
            return i
    raise NotFoundError(f'String {string} not found in File {file.name}'.) 

```